- https://github.com/HridayM25/Physics-Informed-NN/blob/main/main/Burgers'%20Equation.ipynb

In [2]:
import numpy as np 
import torch
import torch.nn as nn 
# from dataCollection import getData
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings("ignore")

torch.manual_seed(42)

In [3]:
import scipy.io
from scipy.interpolate import griddata
import numpy as np
import torch

data = scipy.io.loadmat('burgers_shock.mat')
x = data['x'].flatten()[:, None]
t = data['t'].flatten()[:, None]
usol = np.real(data['usol']).T
X, T = np.meshgrid(x, t)
train = torch.concat([torch.Tensor(X.flatten()[:, None]), torch.Tensor(T.flatten()[:, None])], 1)
X_min = train.min(0)
X_max = train.max(0)

def getData():
    return train, usol, X_min, X_max

X_star, u_star, lb, ub = getData()

In [15]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16,32)
        self.fc3 = nn.Linear(32,1)
    
    def forward(self,x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

In [52]:
class PINN():
    def __init__(self, X, u, lb, ub, physics):
        
        self.lb = torch.tensor(lb).float()
        self.ub = torch.tensor(ub).float()
        self.physics = physics
        
        self.x = torch.tensor(X[:, 0:1], requires_grad=True).float()
        self.t = torch.tensor(X[:, 1:2], requires_grad=True).float()
        self.u = torch.tensor(u).float()
        
        self.network = Network()
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr=0.001)
        
    def makeNetwork(self, x, t):
        X = torch.cat([x,t], dim=1)
        return self.network(X)
    
    def residual(self, x, t):
        u = self.makeNetwork(x, t)
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u),  create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u) , create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs = torch.ones_like(u_x) ,create_graph=True)[0]
        
        return u_t + u*u_x - (0.01/np.pi)*u_xx
    
    def lossResidual(self):
        u_pred = self.makeNetwork(self.x, self.t)
        residual_pred = self.residual(self.x, self.t)
        loss = torch.mean((self.u - u_pred)**2)
        if self.physics == True:  
            loss += torch.mean(residual_pred**2)
        self.optimizer.zero_grad()
        loss.backward()
        return loss
    
    def train(self, n_epochs):
        lossTracker = []
        self.network.train()
        for epoch in range(1, n_epochs + 1):
            u_pred = self.makeNetwork(self.x, self.t)
            residual_pred = self.residual(self.x, self.t)
            loss = torch.mean((self.u - u_pred)**2)

            if self.physics == True:
                loss += torch.mean(residual_pred**2)
            lossTracker.append(loss.item())

            loss.backward()
            # self.optimizer.step()
            self.optimizer.step(self.lossResidual)
            self.optimizer.zero_grad()
            
            if epoch % (n_epochs // 10) == 0:
                print(f"[{epoch:4d}/{n_epochs}] loss: {loss.item():.2e}")
            
        return lossTracker
            
    def predict(self): 
        self.network.eval()
        u = self.makeNetwork(self.x, self.t)
        res = self.residual(self.x, self.t)
        return u.detach().numpy(), res.detach().numpy()

In [53]:
idx = np.random.choice(X_star.shape[0], 2000, replace=False)
X_u_train = X_star[idx, :]
u_train = u_star.flatten()[:, None][idx,:]

In [43]:
model = PINN(X_u_train, u_train, lb[0], ub[0], True)
pinn = model.train(2000)

[ 200/2000] loss: 2.53e-01
[ 400/2000] loss: 1.52e-01
[ 600/2000] loss: 1.34e-01
[ 800/2000] loss: 1.23e-01
[1000/2000] loss: 1.14e-01
[1200/2000] loss: 1.09e-01
[1400/2000] loss: 1.05e-01
[1600/2000] loss: 1.01e-01
[1800/2000] loss: 9.60e-02
[2000/2000] loss: 9.18e-02


In [44]:
model = PINN(X_u_train, u_train, lb[0], ub[0], False)
no_pinn = model.train(1000)

[ 100/1000] loss: 2.50e-01
[ 200/1000] loss: 2.41e-01
[ 300/1000] loss: 1.96e-01
[ 400/1000] loss: 6.66e-02
[ 500/1000] loss: 3.39e-02
[ 600/1000] loss: 3.05e-02
[ 700/1000] loss: 2.84e-02
[ 800/1000] loss: 2.67e-02
[ 900/1000] loss: 2.52e-02
[1000/1000] loss: 2.37e-02


In [51]:
u_pred, res = model.predict()

u_pred.shape, res.mean()

((2000, 1), -0.01629104)